In [1]:
from logic import *

In [2]:
from utils import *

In [3]:
def var(r,c,n):
    return expr(f'X_{r}_{c}_{n}')

In [4]:
def constraint():
    clauses=[]
    for r in range(1,10):
        for c in range(1,10):
                clause=[var(r,c,n) for n in range(1,10)]
                clauses.append(Expr('|',*clause))
    return clauses

In [5]:
def unique_constraint():
    clauses=[]
    for r in range(1,10):
        for c in range(1,10):
            for n in range(1,10):
                for m in range(n+1,10):
                    clauses.append(~var(r,c,n)|~var(r,c,m))
    return clauses

In [6]:
def row_constraint():
    clauses=[]
    for r in range(1,10):
        for n in range(1,10):
            clause=[var(r,c,n) for c in range(1,10)]
            clauses.append(Expr('|',*clause))
            for c1 in range(1,10):
                for c2 in range(c1+1,10):
                    clauses.append(~var(r,c1,n)|~var(r,c2,n))
    return clauses

In [7]:
def col_constraint():
    clauses=[]
    for c in range(1,10):
        for n in range(1,10):
            clause=[var(r,c,n) for r in range(1,10)]
            clauses.append(Expr('|',*clause))
            for r1 in range(1,10):
                for r2 in range(r1+1,10):
                    clauses.append(~var(r1,c,n)|~var(r2,c,n))
    return clauses

In [8]:
def block_constraint():
    clauses=[]
    for br in range(0,3):
        for bc in range(0,3):
            for n in range(1,10):
                cell=[(r,c) for r in range(1+3*br,4+3*br) for c in range(1+3*bc,4+3*bc)]
                clause=[var(r,c,n) for (r,c) in cell]
                clause.append(Expr('|',*clause))
                for i in range(len(cell)):
                    for j in range(i+1,len(cell)):
                        r1,c1=cell[i]
                        r2,c2=cell[j]
                        clauses.append(~var(r1,c1,n)|~var(r2,c2,n))
    return clauses

In [9]:
def puzzle_clauses(puzzle):
    clauses=[]
    for r in range(9):
        for c in range(9):
            n=puzzle[r][c]
            if n!=0:
                clauses.append(var(r+1,c+1,n))
    return clauses

In [10]:
def solver():
    kb=[]
    kb+=constraint()
    kb+=unique_constraint()
    kb+=row_constraint()
    kb+=col_constraint()
    kb+=block_constraint()
    kb+=puzzle_clauses(puzzle)
    model=dpll_satisfiable(Expr('&',*kb))
    if model:
        print("Solved Sudoku PL: ")
        grid=[[0 for _ in range(9)] for _ in range(9)]
        for r in range(1,10):
            for c in range(1,10):
                for n in range(1,10):
                    if model.get(var(r,c,n),False):
                        grid[r-1][c-1]=n
        for row in grid:
            print(row)
    else:
        print("Solution not Found")

In [21]:
puzzle = [
    [0, 1, 0, 0, 4, 0, 0, 5, 0],
    [4, 0, 7, 0, 0, 0, 6, 0, 2],
    [8, 2, 0, 6, 0, 0, 0, 7, 4],
    [0, 0, 0, 0, 1, 0, 5, 0, 0],
    [5, 0, 0, 0, 0, 0, 0, 0, 3],
    [0, 0, 4, 0, 5, 0, 0, 0, 0],
    [9, 6, 0, 0, 0, 3, 0, 4, 5],
    [3, 0, 5, 0, 0, 0, 8, 0, 1],
    [0, 7, 0, 0, 2, 0, 0, 3, 0]
]


In [22]:
solver()

Solved Sudoku PL: 
[6, 1, 9, 7, 4, 2, 3, 5, 8]
[4, 5, 7, 8, 3, 1, 6, 9, 2]
[8, 2, 3, 6, 9, 5, 1, 7, 4]
[2, 3, 6, 4, 1, 9, 5, 8, 7]
[5, 9, 1, 2, 7, 8, 4, 6, 3]
[7, 8, 4, 3, 5, 6, 2, 1, 9]
[9, 6, 2, 1, 8, 3, 7, 4, 5]
[3, 4, 5, 9, 6, 7, 8, 2, 1]
[1, 7, 8, 5, 2, 4, 9, 3, 6]


In [17]:
def Filled(r, c, n, grid):
    return grid[r][c] == n


In [13]:
def is_valid(grid,row,col,num):
    for i in range(9):
        if grid[row][i]==num or grid[i][col]==num:
            return False
    sRow=row-row%3
    sCol=col-col%3
    for i in range(3):
        for j in range(3):
            if grid[sRow+i][sCol+j]==num:
                return False
    return True
    

In [14]:
def fol(grid):
    for r in range(9):
        for c in range(9):
            if grid[r][c]==0:
                for n in range(1,10):
                    if is_valid(grid,r,c,n):
                        grid[r][c]=n
                        if fol(grid):
                            return True
                        grid[r][c]=0
                return False
    return True

In [15]:
def print_grid(grid):
    for r in grid:
        print(r)

In [23]:
if fol(puzzle):
    print("Solved Sudoku FOL: ")
    print_grid(puzzle)
else:
    print("Solution Not Found")


Solved Sudoku FOL: 
[6, 1, 9, 7, 4, 2, 3, 5, 8]
[4, 5, 7, 8, 3, 1, 6, 9, 2]
[8, 2, 3, 6, 9, 5, 1, 7, 4]
[2, 3, 6, 4, 1, 9, 5, 8, 7]
[5, 9, 1, 2, 7, 8, 4, 6, 3]
[7, 8, 4, 3, 5, 6, 2, 1, 9]
[9, 6, 2, 1, 8, 3, 7, 4, 5]
[3, 4, 5, 9, 6, 7, 8, 2, 1]
[1, 7, 8, 5, 2, 4, 9, 3, 6]
